In [31]:
import time
from typing import Any, Dict, List, Tuple, Union

import argparse
import joblib
import tensorflow as tf
from termcolor import cprint
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils.baseline_utils as baseline_utils
#from lstm_train_test_det import *
from lstm_train_test import *

In [32]:
from glob import glob

In [6]:
!ls ../../argoverse

agents_train.npy	       agents_val_rotation.npy	test_obs
agents_train_rotation.npy      agents_val_transi.npy	train
agents_train_small.npy	       argoverse-api.git	train_original
agents_train_small_transi.npy  features			val
agents_train_transi.npy        preprocessing.ipynb	val_original
agents_val.npy		       test


In [33]:
 with open('../../argoverse/agents_train_rotation.npy', 'rb') as f:
    wholetraj = np.load(f)

In [9]:
wholetraj.shape

(205942, 50, 2)

In [12]:
conformal_train = wholetraj[:102971]
conformal_err = wholetraj[102971:]
conformal_train.shape

(102971, 50, 2)

In [13]:
with open('../../argoverse/conformal_train.npy', 'wb') as f:
    np.save(f, conformal_train)
    
with open('../../argoverse/conformal_err.npy', 'wb') as f:
    np.save(f, conformal_err)

In [34]:
device = 'cuda:0'

In [102]:
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)
encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)
encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

model_utils = ModelUtils()

model_path = '../checkpoints/argo_lstm_conformal/LSTM_rollout30.pth.tar'
epoch, rollout_len, _ = model_utils.load_checkpoint(
            model_path, encoder, decoder, encoder_optimizer,
            decoder_optimizer)

=> loading checkpoint '../checkpoints/argo_lstm_conformal/LSTM_rollout30.pth.tar'
=> loaded checkpoint ../checkpoints/argo_lstm_conformal/LSTM_rollout30.pth.tar (epoch: 193, loss: 4.861854553222656)


In [78]:

class LSTMDataset(Dataset):
    """PyTorch Dataset for LSTM Baselines."""
    def __init__(self, wholetraj, shuffle=True):
        normalized = wholetraj

        self.input_data = normalized[:, :20, :]
        self.output_data = normalized[:, 20:, :]
        self.data_size = self.input_data.shape[0]

    def __len__(self):
        return self.data_size

    def __getitem__(self, idx: int
                    ) -> Tuple[torch.FloatTensor, Any, Dict[str, np.ndarray]]:
        return (
            torch.FloatTensor(self.input_data[idx]),
            torch.FloatTensor(
                self.output_data[idx])
        )

In [28]:
model_utils = ModelUtils()

val_dataset = LSTMDataset(conformal_err)
val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=128,
        drop_last=False,
        shuffle=False,
        collate_fn=model_utils.my_collate_fn,
    )

In [67]:
from tqdm import tqdm

In [87]:
# Set to eval mode
encoder.eval()
decoder.eval()
criterion = nn.MSELoss()

rollout_len = 30
all_mse = {}
all_de = []
for i, (_input, target, helpers) in tqdm(enumerate(val_loader)):

    _input = _input.to(device)
    target = target.to(device)

    # Encoder
    batch_size = _input.shape[0]
    input_length = _input.shape[1]
    output_length = target.shape[1]
    input_shape = _input.shape[2]

    # Initialize encoder hidden state
    encoder_hidden = model_utils.init_hidden(
        batch_size,
        encoder.module.hidden_size if use_cuda else encoder.hidden_size)


    # Encode observed trajectory
    for ei in range(input_length):
        encoder_input = _input[:, ei, :]
        encoder_hidden = encoder(encoder_input, encoder_hidden)

    # Initialize decoder input with last coordinate in encoder
    decoder_input = encoder_input
    
    decoder_hidden = encoder_hidden
    output_shape = list(target.shape)
    decoder_outputs = torch.zeros(output_shape).to(device)
    
    mses = torch.zeros(rollout_len).to(device)
    de = []

    for di in range(rollout_len):
        decoder_output, decoder_hidden = decoder(decoder_input,
                                                 decoder_hidden)
        decoder_outputs[:, di, :] = decoder_output
        mses = (decoder_output[:, :2]-target[:, di, :2]).pow(2).sum(axis=-1).pow(0.5)
        if di in all_mse:
            all_mse[di] = np.concatenate((all_mse[di], mses.detach().cpu().numpy()))
        else:
            all_mse[di] = mses.detach().cpu().numpy()

        # Use own predictions as inputs at next step
        decoder_input = decoder_output
        de.append(torch.sqrt((decoder_output[:, 0] - target[:, di, 0])**2 +
                           (decoder_output[:, 1] - target[:, di, 1])**2).detach().cpu().numpy())
    
    all_de.append(de)
    
    if i>60:
        break

61it [00:34,  1.78it/s]


In [84]:
ninetyquant = []
for i in range(30):
    q = np.quantile(all_mse[i], 0.9)
    ninetyquant.append(q)

In [88]:
ninetyquant

[0.9199341535568237,
 0.9753382205963135,
 1.0859079957008362,
 1.2161036133766174,
 1.3877891302108765,
 1.5531651377677917,
 1.752310037612915,
 2.0071715116500854,
 2.247078537940979,
 2.494024634361267,
 2.793949842453003,
 3.1226292848587036,
 3.4556496143341064,
 3.8185853958129883,
 4.143992185592651,
 4.531986236572266,
 4.937242269515991,
 5.381857633590698,
 5.768104076385498,
 6.165144205093384,
 6.541731119155884,
 6.962288856506348,
 7.428401231765747,
 7.888824701309204,
 8.417593002319336,
 8.935048580169678,
 9.438450336456299,
 9.91438627243042,
 10.462123394012451,
 10.978078842163086]

In [92]:
np.array(de).mean()

2.15947

In [75]:
!ls ../../argoverse/agents_val.npy

../../argoverse/agents_val.npy


In [116]:
    
with open('../../argoverse/agents_val_rotation.npy', 'rb') as f:
    test_datase = np.load(f)

In [117]:

test_dataset = LSTMDataset(test_datase)
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=128,
        drop_last=False,
        shuffle=False,
        collate_fn=model_utils.my_collate_fn,
    )

In [188]:
# Set to eval mode
encoder.eval()
decoder.eval()
criterion = nn.MSELoss()

rollout_len = 30
all_coverage = {}
de_test = []
mrses = []
for i, (_input, target, helpers) in tqdm(enumerate(test_loader)):

    _input = _input.to(device)
    target = target.to(device)

    # Encoder
    batch_size = _input.shape[0]
    input_length = _input.shape[1]
    output_length = target.shape[1]
    input_shape = _input.shape[2]

    # Initialize encoder hidden state
    encoder_hidden = model_utils.init_hidden(
        batch_size,
        encoder.module.hidden_size if use_cuda else encoder.hidden_size)


    # Encode observed trajectory
    for ei in range(input_length):
        
        encoder_input = _input[:, ei, :]
        encoder_hidden = encoder(encoder_input, encoder_hidden)

    # Initialize decoder input with last coordinate in encoder
    decoder_input = encoder_input
    decoder_hidden = encoder_hidden
    output_shape = list(target.shape)
    decoder_outputs = torch.zeros(output_shape).to(device)
    
    mses = torch.zeros(rollout_len).to(device)

    for di in range(rollout_len):
        decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
        decoder_outputs[:, di, :] = decoder_output
        mses = (decoder_output[:, :2]-target[:, di, :2]).pow(2).sum(axis=-1).pow(0.5)
        cover = mses < ninetyquant[di]
        
        second_part = 1/0.1 * np.pi * (mses.cpu().detach().numpy()**2 - ninetyquant[di]**2) 
        condition = np.where(second_part<0, 0, second_part)
        mrs = np.pi*ninetyquant[di] ** 2 + condition 
        mrses.append(mrs)
        if di in all_coverage:
            all_coverage[di] = np.concatenate((all_coverage[di], cover.detach().cpu().numpy()))
        else:
            all_coverage[di] = cover.detach().cpu().numpy()

        # Use own predictions as inputs at next step
        decoder_input = decoder_output
        de_test.append(torch.sqrt((decoder_output[:, 0] - target[:, di, 0])**2 +
                           (decoder_output[:, 1] - target[:, di, 1])**2).detach().cpu().numpy())
    if i > 40:
        break

41it [00:23,  1.71it/s]


In [189]:
np.mean(mrses)

211.90213

In [213]:
a =  torch.Tensor(1,40,1).random_(0, 2).long()


In [215]:
a

tensor([[[0],
         [1],
         [0],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0],
         [1],
         [1],
         [1],
         [0],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0],
         [0],
         [1],
         [0],
         [1],
         [1],
         [1],
         [1],
         [0],
         [0],
         [0],
         [0],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0],
         [1],
         [0],
         [0]]])

In [218]:
a.expand((1,40,4)).reshape(1,40,2,2)

RuntimeError: The expanded size of the tensor (2) must match the existing size (40) at non-singleton dimension 2.  Target sizes: [1, 40, 2, 2].  Tensor sizes: [1, 40, 1]

In [ ]:
def mrs():
    second_part = 1/0.1 * np.pi * (mses.pow(2) - ninetyquant[di].pow(2)) 
    condition = np.where(second_part<0, 0, second_part)
    mrs = np.pi*ninetyquant[di].pow(2) + condition


In [97]:
# 309 it

In [172]:
np.array(de_test[:-30]).shape

(9240, 128)

In [165]:
de_array = np.array(de_test)

In [166]:
shapes = [h.shape for h in de_array]

In [167]:
from collections import Counter
Counter(shapes)

Counter({(128,): 9240, (48,): 30})

In [173]:
new_de = np.array(de_test[:-30]).reshape((308,30,128))

In [176]:
fde = np.mean(new_de[:,29,:])
fde

4.6787267

In [145]:
np.mean(np.concatenate(de_test).flatten())


2.159612

In [ ]:
np.mean(np.concatenate(de_test).flatten())

In [122]:
all_coverage[0].mean()

0.9006384272395622

In [126]:
all_coverage[14].mean()

0.926682205107418

In [125]:
all_coverage[29].mean()

0.9276702472638833

In [14]:
!nvidia-smi

Fri Jan 14 01:08:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   44C    P2    61W / 250W |   6363MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 22%   